In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [57]:
dataset=pd.read_csv("CKD.csv")
dataset=pd.get_dummies(dataset,drop_first=True,dtype=int)

In [59]:
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [61]:
independent=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv','wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 
                     'rbc_normal', 'pc_normal','pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes','appet_yes', 'pe_yes', 'ane_yes']]
dependent=dataset[['classification_yes']]

In [63]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [65]:
#standardisation
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [67]:
dataset['classification_yes'].value_counts()

classification_yes
1    249
0    150
Name: count, dtype: int64

In [69]:
from sklearn.svm import SVC

In [71]:
from sklearn.model_selection import GridSearchCV
param_grid={'kernel':['rbf','poly','sigmoid','linear'],'C':[10,100,1000,2000,3000],'gamma':['auto','scale']}
grid= GridSearchCV(SVC(probability=True),param_grid,refit=True,verbose=3,cv=5,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


C:\Users\keert\Anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=5, estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [10, 100, 1000, 2000, 3000],
                         'gamma': ['auto', 'scale'],
                         'kernel': ['rbf', 'poly', 'sigmoid', 'linear']},
             scoring='f1_weighted', verbose=3)

In [73]:
re=grid.cv_results_
grid_predictions=grid.predict(x_test)
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_predictions)
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_predictions)

In [75]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print ("The f1-macro value for best parameter {}:". format(grid.best_params_),f1_macro)
print ("The confusion matrix:\n",cm)
print ("The report:\n",clf_report)

The f1-macro value for best parameter {'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}: 0.9834018801410106
The confusion matrix:
 [[45  0]
 [ 2 73]]
The report:
               precision    recall  f1-score   support

           0       0.96      1.00      0.98        45
           1       1.00      0.97      0.99        75

    accuracy                           0.98       120
   macro avg       0.98      0.99      0.98       120
weighted avg       0.98      0.98      0.98       120



In [77]:
#roc-auc
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.020169,0.005330,0.019828,0.001549,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",0.982221,1.000000,0.982051,1.000000,1.000000,0.992854,0.008752,3
1,0.026189,0.004556,0.018892,0.002395,10,auto,poly,"{'C': 10, 'gamma': 'auto', 'kernel': 'poly'}",1.000000,1.000000,0.964286,1.000000,0.981894,0.989236,0.014311,13
2,0.016689,0.004779,0.018385,0.002124,10,auto,sigmoid,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.982221,1.000000,0.982221,1.000000,1.000000,0.992888,0.008710,1
3,0.011578,0.001643,0.014991,0.002060,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.982221,0.982221,0.982221,0.982051,0.981894,0.982122,0.000132,25
4,0.012649,0.003073,0.015012,0.004993,10,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.982221,1.000000,0.982051,1.000000,1.000000,0.992854,0.008752,3
5,0.021905,0.006444,0.015338,0.002273,10,scale,poly,"{'C': 10, 'gamma': 'scale', 'kernel': 'poly'}",1.000000,1.000000,0.964286,1.000000,0.981894,0.989236,0.014311,13
6,0.013603,0.004672,0.014333,0.002804,10,scale,sigmoid,"{'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.982221,1.000000,0.982221,1.000000,1.000000,0.992888,0.008710,1
7,0.013221,0.005246,0.016133,0.002973,10,scale,linear,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}",0.982221,0.982221,0.982221,0.982051,0.981894,0.982122,0.000132,25
8,0.009905,0.002076,0.010859,0.001623,100,auto,rbf,"{'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}",0.982221,1.000000,0.982051,0.982051,1.000000,0.989265,0.008766,5
9,0.016762,0.002830,0.016905,0.001661,100,auto,poly,"{'C': 100, 'gamma': 'auto', 'kernel': 'poly'}",0.964572,1.000000,0.982051,1.000000,0.981894,0.985703,0.013290,17


In [79]:
grid.predict([[15,60,3,0,86,15,0.6,138,4,11,33,7700,3.8,0,0,0,0,1,1,0,0,1,1,0,1,0,0]])

array([0])

In [81]:
h=roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])
print(h)

0.9997037037037036


In [83]:
grid.predict([[15,80,0,0,93,17,0.9,136,3.9,16.7,50,6200,5.2,1,0,0,0,1,1,0,0,0,0,0,1,0,0]])

array([0])